# Processamento de Dados de Vendas (2022-2024)
Este notebook é dedicado ao tratamento abrangente de dados de vendas abrangendo o período de 2022 a 2024. Sua função principal é padronizar e corrigir os dados brutos de acordo com as regras comerciais estabelecidas, garantindo a integridade e a confiabilidade do conjunto de dados.

In [436]:
# Defininindo o arquivo de trabalho
arquivo = '../data/raw/vendas_22-24.csv'

In [437]:
# Importação das bibliotecas utilizadas.
import pandas as pd
import statistics as sts

In [438]:
# Carregamento do arquivo de dados a ser utilizado.
dados = pd.read_csv(arquivo, delimiter=';')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             9672 non-null   object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       9691 non-null   float64
 5   Preço Unitário   9688 non-null   float64
 6   Total Venda      7979 non-null   float64
 7   Cliente_ID       9732 non-null   float64
 8   Forma_Pagamento  9684 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


In [439]:
# Verificando o padrão de colunas
dados.head()

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
0,2022-01-18,Loja B,Refrigerante,Bebidas,3.0,7.50,22.50,6727.0,Dinheiro
1,2024-12-29,Loja B,Leite,Alimentos,5.0,14.74,73.70,2120.0,Transferência
2,2023-03-21,Loja C,Refrigerante,Bebidas,2.0,NaN,72.60,5353.0,Transferência
3,2022-08-03,Loja C,Feijão,Alimentos,4.0,26.75,107.00,6835.0,Transferência
4,2022-06-29,Loja C,Feijão,Alimentos,7.0,30.47,213.29,5824.0,Dinheiro


In [440]:
# contagem de valores nulos.
dados.isnull().sum()

Data                  0
Loja                328
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID          268
Forma_Pagamento     316
dtype: int64

### Regra de negócio Loja

In [441]:
# Verificação dos dados de loja
dados.groupby('Loja').size()

Loja
Loja A    3238
Loja B    3215
Loja C    3219
dtype: int64

In [442]:
# Preenche o valor da loja com o valor de maior recorrência
dados.fillna({'Loja': 'Loja A'}, inplace=True)

### Regra de negócio Cliente

In [443]:
dados['Cliente_ID'].isnull().sum()

np.int64(268)

In [444]:
dados['Cliente_ID'].loc[dados['Cliente_ID'].isnull()] = 0000

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\510108161.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dados['Cliente_ID'].loc[dados['Cliente_ID'].isnull()] = 0000
C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\510108161.py

In [445]:
dados['Cliente_ID'].isnull().sum()

np.int64(0)

### Regra de negócio Quantidade

In [446]:
# Nova contagem de nulos
dados.isnull().sum()

Data                  0
Loja                  0
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID            0
Forma_Pagamento     316
dtype: int64

In [447]:
# Calculo da mediana para preenchimento de valores incorretos ou nulos
medianaQtd = sts.median(dados['Quantidade'])
medianaQtd

2.0

In [448]:
# Preenchimento de valores nulos
dados['Quantidade'].fillna(medianaQtd, inplace=True)

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\3798690500.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['Quantidade'].fillna(medianaQtd, inplace=True)


In [449]:
# Filtro de valores fora da regra de negócio
dados['Quantidade'].loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

11     -4.0
67     -5.0
77     -4.0
166    -1.0
179    -5.0
       ... 
9867   -4.0
9873   -1.0
9888   -2.0
9985   -5.0
9987   -1.0
Name: Quantidade, Length: 367, dtype: float64

In [450]:
# Correção de valores incorretos com base na regra de negócio
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)] = medianaQtd

In [451]:
# Nova verificação de valores fora da regra
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento


In [452]:
dados = dados.astype({
    'Data': 'datetime64[us]',
    'Loja': 'string',
    'Produto': 'string',
    'Categoria': 'string',
    'Quantidade': 'int64',
    'Cliente_ID': 'int64',
    'Forma_Pagamento': 'string'
})

In [453]:
# Contagem de nulos
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             10000 non-null  datetime64[us]
 1   Loja             10000 non-null  string        
 2   Produto          10000 non-null  string        
 3   Categoria        10000 non-null  string        
 4   Quantidade       10000 non-null  int64         
 5   Preço Unitário   9697 non-null   float64       
 6   Total Venda      8346 non-null   float64       
 7   Cliente_ID       10000 non-null  int64         
 8   Forma_Pagamento  9696 non-null   string        
dtypes: datetime64[us](1), float64(2), int64(2), string(4)
memory usage: 703.3 KB


### Regra de negócio Preço

In [454]:
mediaPreco = dados['Preço Unitário'].mean()
mediaPreco = round(mediaPreco, 2)
mediaPreco

np.float64(22.42)

In [455]:
dados['Preço Unitário'].loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500)] = mediaPreco

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\937395872.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dados['Preço Unitário'].loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500)] = mediaPreco
C:\Users\jamsilva

In [456]:
dados['Preço Unitário'].fillna(mediaPreco, inplace=True)
dados.loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500.0)] = mediaPreco

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\1929562576.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['Preço Unitário'].fillna(mediaPreco, inplace=True)


In [457]:
dados['Preço Unitário'].describe()

count    10000.000000
mean        24.661908
std         13.959693
min          1.500000
25%         13.157500
50%         22.595000
75%         36.330000
max         50.000000
Name: Preço Unitário, dtype: float64

### Regra Total da Venda

In [458]:
dados['Total Venda'].describe()

count    8346.000000
mean      135.277260
std       114.645145
min         1.540000
25%        39.600000
50%       103.600000
75%       206.727500
max       499.300000
Name: Total Venda, dtype: float64

In [ ]:
dados.loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
2,2023-03-21 00:00:00.000000,Loja C,Refrigerante,Bebidas,2,22.42,72.60,5353,Transferência
5,2023-12-27 00:00:00.000000,Loja A,Refrigerante,Bebidas,9,15.64,NaN,8287,Pix
10,2022-03-29 00:00:00.000000,Loja B,Pasta de Dente,Higiene,2,31.21,NaN,1659,Dinheiro
11,1970-01-01 00:00:00.000002,2.0,2.0,2.0,2,2.00,2.00,2,2.0
14,2024-06-01 00:00:00.000000,Loja B,Arroz,Alimentos,5,10.27,51.35,4418,Pix
...,...,...,...,...,...,...,...,...,...
9987,1970-01-01 00:00:00.000002,2.0,2.0,2.0,2,2.00,2.00,2,2.0
9988,2024-10-20 00:00:00.000000,Loja A,Detergente,Limpeza,2,11.74,NaN,9310,Dinheiro
9989,2024-11-08 00:00:00.000000,Loja C,Cerveja,Bebidas,5,49.33,246.65,5255,Dinheiro
9991,2023-05-04 00:00:00.000000,Loja A,Pasta de Dente,Higiene,6,35.05,NaN,9151,Transferência


In [462]:
dados['Total Venda'].loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])] = dados['Quantidade'] * dados['Preço Unitário']

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_20240\1496579656.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dados['Total Venda'].loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])] = dados['Quantidade'] * dad

In [463]:
dados.loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
